In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

### Image Addition

In [ ]:
# There is a difference between OpenCV addition and Numpy addition.
#OpenCV addition is a saturated operation while Numpy addition is
#a modulo operation.
x = np.uint8([250])
y = np.uint8([10])
print(cv2.add(x,y)) # 250+10 = 260 => 255

print(x+y)          # 250+10 = 260 % 256 = 4


### Image blending

In [ ]:
#This is also image addition, but different weights are given to
#images so that it gives a feeling of blending or transparency.
#Images are added as per the equation below:
#g(x) = (1-a)f0(x) = af1(x)

img1 = cv2.imread('Data/ml.jpg')
img2 = cv2.imread('Data/opencv_logo.png')

#dst = cv2.add(img1,img2)
#plt.imshow(dst)

dst = cv2.addWeighted(img1,0.8,img2,0.1,0)
plt.imshow(dst)





#cv2.imshow('dst',dst)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

### Bitwise Operations

In [ ]:
# Bitwise Operations


In [26]:
# Load two images
# suppose opencv_logo is not rectangular
img1 = cv2.imread('Data/messi5.jpg')
img2 = cv2.imread('Data/opencv_logo.png')

# I want to put logo on top-left corner, So I create a ROI
rows,cols,channels = img2.shape
roi = img1[0:rows, 0:cols ]

# Now create a mask of logo and create its inverse mask also
img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
# any value less than 10 is 0 and larger than 10 is 255
ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
mask_inv = cv2.bitwise_not(mask)


# Now black-out the area of logo in ROI
img1_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)

# Take only region of logo from logo image.
img2_fg = cv2.bitwise_and(img2,img2,mask = mask)

#cv2.imshow('res',img1_bg)
#cv2.imshow('res',img2_fg)

# Put logo in ROI and modify the main image
dst = cv2.add(img1_bg,img2_fg)
img1[0:rows, 0:cols ] = dst

cv2.imshow('res',img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

### More samples

## Alpha blending 

In [47]:
src1 = cv2.imread('data/lena.jpg')
src2 = cv2.imread('data/rocket.jpg')
print(src1.shape)
print(src2.shape)

src2 = cv2.resize(src2, src1.shape[1::-1])
print(src2.shape)


(225, 400, 3)
(427, 640, 3)
(225, 400, 3)


In [48]:
#x=np.arange(18).reshape(3,2,3)
#print(x.shape)
#print(x)
#y = x[1::-1]
#print(y.shape)
#print(y)


In [49]:
dst = cv2.addWeighted(src1, 0.5, src2, 0.5, 0)

cv2.imwrite('saved-images/opencv_add_weighted.jpg', dst)

True

In [50]:
dst = cv2.addWeighted(src1, 0.5, src2, 0.2, 128)

cv2.imwrite('saved-imaages/opencv_add_weighted_gamma.jpg', dst)

True

In [51]:
src2 = cv2.imread('data/horse_r.png')

src2 = cv2.resize(src2, src1.shape[1::-1])

print(src2.shape)
# (225, 400, 3)

print(src2.dtype)
# uint8

dst = cv2.bitwise_and(src1, src2)

cv2.imwrite('saved-images/opencv_bitwise_and.jpg', dst)

(225, 400, 3)
uint8


True

## Alpha blending with NumPy

In [54]:
# Here, image files are read as NumPy array ndarray using Pillow. 
#Resize is also done by the method of Pillow.

import numpy as np
from PIL import Image

src1 = np.array(Image.open('data/lena.jpg'))
src2 = np.array(Image.open('data/rocket.jpg').resize(src1.shape[1::-1], Image.BILINEAR))

print(src1.dtype)
# uint8

dst = src1 * 0.5 + src2 * 0.5

print(dst.dtype)
# float64

Image.fromarray(dst.astype(np.uint8)).save('saved-images/numpy_image_alpha_blend.jpg')

uint8
float64


### Masking with NumPy

In [55]:
# The arithmetic operations of ndarrays of the same shape are 
#operations for each pixel at the same position.
import numpy as np
from PIL import Image

src = np.array(Image.open('data/lena.jpg'))
mask = np.array(Image.open('data/horse_r.png').resize(src.shape[1::-1], Image.BILINEAR))

print(mask.dtype, mask.min(), mask.max())
# uint8 0 255

mask = mask / 255

print(mask.dtype, mask.min(), mask.max())
# float64 0.0 1.0

dst = src * mask

Image.fromarray(dst.astype(np.uint8)).save('saved-images/numpy_image_mask.jpg')

# If mask image is grey scale (2D) then you must fix it first
#mask = mask.reshape(*mask.shape, 1)
##or mask = mask[:, :, np.newaxis]
#print(mask.shape)
## (225, 400, 1)

#dst = src * mask


uint8 0 255
float64 0.0 1.0


## Mask image creation by OpenCV drawing

In [61]:
import cv2
import numpy as np

src = cv2.imread('data/lena.jpg')

mask = np.zeros_like(src)

print(mask.shape)
# (225, 400, 3)

print(mask.dtype)
# uint8

cv2.rectangle(mask, (50, 50), (100, 200), (255, 255, 255), thickness=-1)
cv2.circle(mask, (200, 100), 50, (255, 255, 255), thickness=-1)
cv2.fillConvexPoly(mask, np.array([[330, 50], [300, 200], [360, 150]]), (255, 255, 255))

cv2.imwrite('saved-images/opencv_draw_mask.jpg', mask)

#When smoothing (blurring) processing is performed
#  (kernel size needs to be odd) and Gaussian SD if 0 it is calculated auto
mask_blur = cv2.GaussianBlur(mask, (51, 51), 0)

cv2.imwrite('saved-images/opencv_draw_mask_blur.jpg', mask_blur)

dst = src * (mask_blur / 255)
cv2.imwrite('saved-images/opencv_draw_mask_blur_result.jpg', dst)


(225, 400, 3)
uint8


True